In [124]:
import numpy as np
import matplotlib.pyplot as plt


In [125]:
GRID = {'min':-5,
        'max':5,
        'steps':10}

def space(grid):
    try:
        return np.linspace(grid['min'],grid['max'],grid['steps'])
    except:
        return np.linspace(grid[0],grid[1],grid[2])
    
def plot_field(u,v): # u and v are functions
    x,y = np.meshgrid(np.linspace(GRID['min'],GRID['max'],GRID['steps']),np.linspace(GRID['min'],GRID['max'],GRID['steps']))
    plt.quiver(x,y,u(x,y),v(x,y))
    plt.show()

# u is dx, v is dy
u = lambda x,y: 2*x
v = lambda x,y: x+y

#plot_field(u,v)


In [126]:
def square(n,density=1):
    return space([-n,n,n*density*2])

def thetadot(x):
    return 3+np.cos(2*x)

def theta2dot(x,y):
    return -2*y*np.sin(2*x)

u = lambda x,y: y
v = lambda x,y: theta2dot(x,y)

#plot_field(u,v)


In [127]:
# colorizing stuff
import matplotlib.cm as cm
from matplotlib.colors import Normalize
colormap = cm.Reds

x,y = np.meshgrid(space([-7,7,10]),space([0,10,10]))

u = y
v = theta2dot(x,y)

colors = np.hypot(u,v)
u,v = u/colors,v/colors
#plt.quiver(x,y,u,v,colors,cmap=colormap)
#plt.show()

<ipython-input-127-2dd755cf4457>:12: RuntimeWarning: invalid value encountered in true_divide
  u,v = u/colors,v/colors


In [128]:
# a Field class to store all the settings
import sklearn.preprocessing as prep

class Field():
    zero_fn = lambda v: (v[0]+v[1])*0 # required to fill arrays properly
    def __init__(self,xdot=zero_fn,ydot=zero_fn,mag_scale=False,xlims=[-5,5],ylims=[-5,5],density=1):
        self.xdot = xdot # function to get xdot
        self.ydot = ydot # function to get ydot
        self.mag_scale = mag_scale
        self.xlims = xlims
        self.ylims = ylims
        self.density = density
        ar = np.arange(-10,10,2)
        self.grid = np.array(np.meshgrid(ar,ar)).T.reshape([100,2])
        self.xycomps = None
        self.arrows=True
    def add_diffeq(self,xdot,ydot=zero_fn): # if no y is given, vectors will only be in the x direction--a 1d system
        self.xdot = xdot
        self.ydot = ydot
    def calc_vectors(self,xdot=None,ydot=None):
        if xdot is not None:
            self.xdot = xdot
        if ydot is not None:
            self.ydot = ydot
        self.xycomps = np.array([self.xdot(self.grid.T),self.ydot(self.grid.T)]).T[:,:,0]
        if not self.mag_scale: # make all arrows the same length
            self.xycomps = prep.normalize(self.xycomps)
            
    def plot(self,xdot=None,ydot=None): 
        if xdot is None:
            xdot = self.xdot
        if ydot is None:
            ydot = self.ydot
        self.calc_vectors(xdot=xdot,ydot=ydot)
        comps = self.xycomps
        x = self.grid[:,0]
        y = self.grid[:,1]
        plt.quiver(x,y,comps[:,0],comps[:,1],width=.004)
        plt.show()
    